In [ ]:
# Imports
import sys
sys.path.append('../')

import datetime
import warnings
import pandas as pd
from scripts.main import laad_data, bekijk_ziekteverzuim, voorspel_met_voortschrijdend_gemiddelde, voorspel_met_regressie, pas_voortschrijdend_gemiddelde_toe, pas_regressie_toe, onderzoek_afwijkingen, bereken_metrieken, voorspel, opties_berekenen, optie_1, optie_2, optie_3


## Settings
# settings for pandas
pd.set_option("display.max.columns",None) # alle kolommen tonen
pd.set_option("display.max.rows",500)    # eerste 500 rijen tonen
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # floats output tot 3 decimalen
pd.set_option('display.max_colwidth', None)
datum_vandaag = datetime.datetime.now().strftime("%Y-%m-%d")
# Disable warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv(filepath_or_buffer='../data/Timeseries.csv', sep=';')
# onderwerp = kies_onderwerp()

In [ ]:
data.head()

In [ ]:
cut_off_date = '2023-05-15'
data['Datum'] = pd.to_datetime(data['Datum'])
data.set_index('Datum', inplace=True)

In [ ]:
data.tail()

In [ ]:
def voorspel_met_voortschrijdend_gemiddelde_fixed(data,
        onderwerp='Ziekteverzuim',
        voorspellen_tot_datum='2025-01-01',
        vensterlengte=7,
        verschuiving=0,
        jaarlijks_patroon=None,
        wekelijks_patroon=None,
        graad=None,
        model=None,
        zie_traintest_periodes=False):
    
    _data = data.copy()
    vanaf_datum_train_periode = _data.index.min()
    maximum_data_dataset = datetime.datetime(2024,12,31)
    date_yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
    maximum_date = min(date_yesterday, maximum_data_dataset)
    tot_datum_train_periode = maximum_date
    vanaf_datum_test_periode = maximum_date
    tot_datum_test_periode = voorspellen_tot_datum

    df = pas_voortschrijdend_gemiddelde_toe(
            data=_data,
            onderwerp=onderwerp,
            vensterlengte=vensterlengte,
            verschuiving=verschuiving,
            vanaf_datum_train_periode = vanaf_datum_train_periode,
            tot_datum_train_periode = '2023-05-15',
            vanaf_datum_test_periode = '2023-05-15',
            tot_datum_test_periode = '2024-05-15',
            zie_traintest_periodes=zie_traintest_periodes,
            plot=False
        )
    return df

def voorspel_met_regressie_fixed(data,
        onderwerp='Ziekteverzuim',
        voorspellen_tot_datum='2025-01-01',
        vensterlengte=None,
        verschuiving=None,
        jaarlijks_patroon=False,
        wekelijks_patroon=False,
        graad=1,
        model=None,
        zie_traintest_periodes=False):

    _data = data.copy()
    vanaf_datum_train_periode = _data.index.min()
    maximum_data_dataset = datetime.datetime(2024,12,31)
    date_yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
    maximum_date = min(date_yesterday, maximum_data_dataset)
    tot_datum_train_periode = maximum_date
    vanaf_datum_test_periode = maximum_date
    tot_datum_test_periode = voorspellen_tot_datum

    df = pas_regressie_toe(
            data=_data,
            onderwerp=onderwerp,
            jaarlijks_seizoenspatroon=jaarlijks_patroon,
            wekelijks_seizoenspatroon=wekelijks_patroon,
            graad=graad,
            vanaf_datum_train_periode = vanaf_datum_train_periode,
            tot_datum_train_periode = '2023-05-15',
            vanaf_datum_test_periode = '2023-05-15',
            tot_datum_test_periode = '2024-05-15',
            zie_traintest_periodes=zie_traintest_periodes,
            plot=False
        )

    return df   

In [ ]:
df_metrics_vg = pd.DataFrame()
for onderwerp in ['Ziekteverzuim']:
    for vensterlengte in [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 73, 365]:
        for verschuiving in [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 73, 365, 730]:
            voortschrijdend_gemiddelde = voorspel_met_voortschrijdend_gemiddelde_fixed(data=data, 
                                                                            onderwerp=onderwerp, 
                                                                            vensterlengte=vensterlengte, 
                                                                            verschuiving=verschuiving, 
                                                                            voorspellen_tot_datum='2025-01-01')
            metrieken = bereken_metrieken(list_of_dfs=[voortschrijdend_gemiddelde], 
                                        onderwerp=onderwerp,
                                        print_statement=False)
            metrieken['onderwerp'] = onderwerp
            metrieken['vensterlengte'] = vensterlengte
            metrieken['verschuiving'] = verschuiving
            df_metrics_vg = pd.concat([df_metrics_vg, metrieken])

In [ ]:
regressie = pas_regressie_toe(data=data, onderwerp=onderwerp)

df_metrics_lr = pd.DataFrame()
for onderwerp in ['Ziekteverzuim']:
    for jaarlijks_seizoenspatroon in [True, False]:
        for wekelijks_seizoenspatroon in [True, False]:
            for graad in range(1,24):
                regressie = voorspel_met_regressie_fixed(data=data, 
                                            onderwerp=onderwerp,
                                            jaarlijks_patroon=jaarlijks_seizoenspatroon,
                                            wekelijks_patroon=wekelijks_seizoenspatroon,
                                            graad=graad,
                                            voorspellen_tot_datum='2025-01-01')
                metrieken = bereken_metrieken(list_of_dfs=[regressie], 
                                            onderwerp=onderwerp,
                                            print_statement=False)
                metrieken['onderwerp'] = onderwerp
                metrieken['jaarlijks_seizoenspatroon'] = jaarlijks_seizoenspatroon
                metrieken['wekelijks_seizoenspatroon'] = wekelijks_seizoenspatroon
                metrieken['graad'] = graad
                df_metrics_lr = pd.concat([df_metrics_lr, metrieken])

In [ ]:
df_metrics_vg.sort_values(by='Juistheid', ascending=False)

In [ ]:
df_metrics_lr.sort_values(by='Juistheid', ascending=False)